In [154]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime, timezone, timedelta
import pytz
import calendar
import requests

In [227]:
#read in dark sky API hourly forecast for the next 48 hours
#uses API key + latitude and longitude for the same monitor site ozone data is collected
#excludes unnecessary information
response=requests.get('https://api.darksky.net/forecast/***YOUR API KEY ****/34.066,-118.22688?exclude=[currently,minutely,daily,alerts,flags]')
date_json = response.json()
date_json['hourly']['data'][0]

#extract the hourly data only and parse through to find the values needed for model
day = date_json['hourly']['data']
all_hours=[]

for hour in day:
    t= hour['time']
    temp=hour['temperature']
    press=hour['pressure']
    wind=hour['windSpeed']
    humidity=hour['humidity']
    all_hours.append([t, temp, press, wind, humidity])

#convert utc time to local time in LA and format nicely
for row in all_hours:
    row[0] = datetime.fromtimestamp(row[0], pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S')

#turn list of lists into a dataframe and label columns
weather_pred = pd.DataFrame.from_records(all_hours, columns = ['time', 'temp', 'pressure', 'wind', 'humidity'])

#remove current weather because that information comes from ozone monitor source
weather_pred = weather_pred.iloc[1:]

#convert to datetime, set as index, and set base to current time for resampling in 8 hour intervals
weather_pred['time'] = pd.to_datetime(weather_pred['time'])
weather_pred.set_index('time', inplace = True)
date_base = weather_pred.index[0]
hour_base = int(date_base.strftime('%H'))
weather_pred_8hr = weather_pred.resample('8H', base = hour_base).max()

#change units to match model input
weather_pred_8hr['wind'] = weather_pred_8hr['wind'] * 1.94384
weather_pred_8hr['humidity'] = weather_pred_8hr['humidity'] * 100
#rearrange column order to make it easier to format input arry
weather_pred_8hr= weather_pred_8hr[['temp', 'pressure', 'humidity', 'wind']]
weather_pred_8hr

,temp,pressure,humidity,wind
time,,,,
2019-10-03 19:00:00,68.69,1014.78,74.0,6.239726
2019-10-04 03:00:00,68.99,1016.07,76.0,8.008621
2019-10-04 11:00:00,81.80,1015.88,34.0,17.883328
2019-10-04 19:00:00,73.72,1016.64,51.0,8.630650
2019-10-05 03:00:00,73.58,1017.85,29.0,8.397389
2019-10-05 11:00:00,87.09,1017.58,21.0,12.771029


In [214]:
#get current time to be able to properly request the correct data files from Air Quality API
time_request = datetime.utcnow()

#Air Quality API for today's hourly updated weather/ozone measurements
front_url = 'https://s3-us-west-1.amazonaws.com//files.airnowtech.org/airnow/today/HourlyData_'
back_url = ".dat"
past_list = []

#loop through all 24 hours of previous data
for i in range(2, 25):
    #data sometimes slow to be available from the site and that causes errors - 
    #begin with deleting two hours and work up to 24 hours back
    time_change = time_request - timedelta(hours=i)
    time_url = (time_change.strftime('%Y%m%d%H'))
    url_combo = front_url + time_url + back_url
    data_file = pd.read_csv(url_combo, sep = '|',
                       names = ['Date', 'Time', 'Site num', 'Site name', 'GMT offset', 'Parameter name', 
                               'Units', 'Value', 'Agency' ])
    la = data_file[data_file['Site name'] == 'Los Angeles - N. Mai'] 
    la_need = la[(la['Parameter name'] == 'OZONE') | (la['Parameter name'] == 'TEMP')| 
             (la['Parameter name'] == 'WS')| (la['Parameter name'] == 'BARPR') |  
             (la['Parameter name'] == 'RHUM')]
    past_list.append([la_need.iloc[1,0], la_need.iloc[0,1], la_need.iloc[0,7],la_need.iloc[1,7],
                      la_need.iloc[2,7],la_need.iloc[3,7], la_need.iloc[4,7]])

#convert list of lists to dataframe and name columns
past_df = pd.DataFrame.from_records(past_list, columns = ['date', 'time', 'pressure', 'ozone', 'humidity', 'temp', 'wind'])

#combine date and time, convert to datetime object, and set as index
past_df['Time'] = pd.to_datetime(past_df['date'] + ' ' + past_df['time'])
past_df.set_index('Time', inplace = True)

#remove old date and time columns
past_df.drop(['date', 'time'], axis = 1, inplace = True)

#parse out hour to set the base for 8 hour resampling
date_base = past_df.index[0]
hour_base = int(date_base.strftime('%H'))
past_df_8hr = past_df.resample('8H', base = hour_base).max()

#convert to correct units that match model input 
past_df_8hr['ozone'] = past_df_8hr['ozone']/1000
past_df_8hr['temp'] = past_df_8hr['temp']*9/5 + 32
past_df_8hr['wind'] = past_df_8hr['wind']*1.94384

#rearrange column order to make it easier to format input arry
past_df_8hr= past_df_8hr[['temp', 'ozone', 'pressure', 'humidity', 'wind']]
past_df_8hr

### Need to have AWS run this every hour so it's pre-loaded because it's very slow
# then export in csv of little, formatted, df. right now use csv placeholder
past_df_8hr.to_csv('past_df_8hr.csv')
past_df_8hr

,temp,ozone,pressure,humidity,wind
Time,,,,,
2019-10-02 14:00:00,80.24,0.061,1001.6,47.3,8.747280
2019-10-02 22:00:00,77.90,0.062,1000.2,62.6,8.941664
2019-10-03 06:00:00,63.68,0.010,1000.3,72.0,3.304528
2019-10-03 14:00:00,58.28,0.003,1000.9,61.1,3.110144


In [228]:
#for main applcation, have AWS get new ozone data every hour #and sorth throught to export csv file
# because it runs very slowly and I want it to be fast when someone clicks in the app
#this is practice to make sure the code is perfect for the app
past_df_8hr = pd.read_csv('past_df_8hr.csv')
past_df_8hr.set_index('Time', inplace = True)
past_df_8hr

,temp,ozone,pressure,humidity,wind
Time,,,,,
2019-10-02 14:00:00,80.24,0.061,1001.6,47.3,8.747280
2019-10-02 22:00:00,77.90,0.062,1000.2,62.6,8.941664
2019-10-03 06:00:00,63.68,0.010,1000.3,72.0,3.304528
2019-10-03 14:00:00,58.28,0.003,1000.9,61.1,3.110144


In [229]:
#add in month, day, hour for current time
#there has to be a better way - FIX LATER
my_date = datetime.now(pytz.timezone('US/Pacific'))
data = [{'Aug': 0, 'Dec': 0, 'Feb':0,'Jan': 0, 'July': 0, 'June':0, 'March':0,
         'May': 0, 'Nov': 0, 'Oct':0,'Sept': 0, '8': 0, '16':0,
        'Mon': 0, 'Sat': 0, 'Sun':0, 'Thur': 0, 'Tue': 0, 'Wed':0}]
date_info = pd.DataFrame(data)

#Month fill in
if my_date.strftime('%B') == 'September':
    date_info['Sept'] = 1
elif my_date.strftime('%B') == 'October':
    date_info['Oct'] = 1
elif my_date.strftime('%B') == 'November':
    date_info['Nov'] = 1
elif my_date.strftime('%B') == 'December':
    date_info['Dec'] = 1
elif my_date.strftime('%B') == 'January':
    date_info['Jan'] = 1
elif my_date.strftime('%B') == 'February':
    date_info['Feb'] = 1
elif my_date.strftime('%B') == 'March':
    date_info['March'] = 1
elif my_date.strftime('%B') == 'May':
    date_info['May'] = 1
elif my_date.strftime('%B') == 'June':
    date_info['June'] = 1
elif my_date.strftime('%B') == 'July':
    date_info['July'] = 1
elif my_date.strftime('%B') == 'August':
    date_info['Aug'] = 1

#Hour range fill in
if (int(my_date.strftime('%H')) >= 16) & (int(my_date.strftime('%H')) <= 23):
    date_info['16'] = 1
elif (int(my_date.strftime('%H')) >= 8) & (int(my_date.strftime('%H')) <= 15):
    date_info['8'] = 1
    
#Day of week fill in
if my_date.strftime('%A') == 'Monday':
    date_info['Mon'] = 1
elif my_date.strftime('%A') == 'Tuesday':
    date_info['Tue'] = 1
elif my_date.strftime('%A') == 'Thursday':
    date_info['Thur'] = 1
elif my_date.strftime('%A') == 'Wednesday':
    date_info['Wed'] = 1
elif my_date.strftime('%A') == 'Saturday':
    date_info['Sat'] = 1
elif my_date.strftime('%A') == 'Sunday':
    date_info['Sun'] = 1

date_info   

,Aug,Dec,Feb,Jan,July,June,March,May,Nov,Oct,Sept,8,16,Mon,Sat,Sun,Thur,Tue,Wed
0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0


In [230]:
#assemble all values into np arrary in the same order as model input
ordered_input = np.array([past_df_8hr.iloc[3].values, past_df_8hr.iloc[2].values, past_df_8hr.iloc[1].values, past_df_8hr.iloc[0].values, 
                        weather_pred_8hr.iloc[0].values, weather_pred_8hr.iloc[1].values, weather_pred_8hr.iloc[2].values, 
                        weather_pred_8hr.iloc[3].values, weather_pred_8hr.iloc[4].values, weather_pred_8hr.iloc[5].values,
                        date_info.iloc[0].values])
#arrays within array, concatenate to format appropriately for predictions from model fit
ordered_input = np.concatenate(ordered_input).ravel()
ordered_input

array([5.82800000e+01, 3.00000000e-03, 1.00090000e+03, 6.11000000e+01,
       3.11014400e+00, 6.36800000e+01, 1.00000000e-02, 1.00030000e+03,
       7.20000000e+01, 3.30452800e+00, 7.79000000e+01, 6.20000000e-02,
       1.00020000e+03, 6.26000000e+01, 8.94166400e+00, 8.02400000e+01,
       6.10000000e-02, 1.00160000e+03, 4.73000000e+01, 8.74728000e+00,
       6.86900000e+01, 1.01478000e+03, 7.40000000e+01, 6.23972640e+00,
       6.89900000e+01, 1.01607000e+03, 7.60000000e+01, 8.00862080e+00,
       8.18000000e+01, 1.01588000e+03, 3.40000000e+01, 1.78833280e+01,
       7.37200000e+01, 1.01664000e+03, 5.10000000e+01, 8.63064960e+00,
       7.35800000e+01, 1.01785000e+03, 2.90000000e+01, 8.39738880e+00,
       8.70900000e+01, 1.01758000e+03, 2.10000000e+01, 1.27710288e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [231]:
#read in data for model and format
df = pd.read_csv('ozone_8hr_lags.csv')
#df = pd.read_csv('uncertainty.csv')
df.rename(columns = {"Unnamed: 0": "Date"}, inplace = True) 
df = df.set_index('Date')
df.index = pd.to_datetime(df.index)
df.dropna(inplace = True) 
df = pd.get_dummies(df, columns = ['Month', 'Hour', 'Day'], drop_first = True)
reg = LinearRegression()
X = df.drop(['Ozone+8', 'Ozone+16', 'Ozone+24', 'Ozone+32', 'Ozone+40', 'Ozone+48'], axis = 1)
#normalize in a way that I can re-use normalization conditions for future prediction values
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)
y = df[['Ozone+8', 'Ozone+16', 'Ozone+24', 'Ozone+32', 'Ozone+40', 'Ozone+48']]
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size = 0.3, random_state = 21)
reg.fit(X_train, y_train)
# y_pred = reg.predict(X_test)
# r2_score(y_test, y_pred)
# mean_squared_error(y_test, y_pred)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [232]:
#normalize model prediction values and then get prediction
ordered_input_scaled = scaler.transform(ordered_input.reshape(1, -1))
pred_app = reg.predict(ordered_input_scaled)
pred_app

array([[0.03562997, 0.01690379, 0.05642207, 0.03312664, 0.01392009,
        0.05852899]])

In [241]:
#format output to nice dataframe that I can work with for more formatting
output = pd.DataFrame(data=pred_app[:,:])
output.columns = ['+8 hours', '+16 hours', '+24 hours', '+32 hours', '+40 hours', '+48 hours']
output = output.transpose()
output = output.reset_index()
output['risk'] = "low"
output.columns = ['time', 'level', 'risk']
output['level'] = round(output['level'],3)
output

,time,level,risk
0,+8 hours,0.036,low
1,+16 hours,0.017,low
2,+24 hours,0.056,low
3,+32 hours,0.033,low
4,+40 hours,0.014,low
5,+48 hours,0.059,low


In [244]:
#in the app, users with enter these input values and this will determine their personalized risk
#for now, use these placeholder inputs
sensitivity = "high"
intensity = "high"
duration = "long"

######################################################
######  All combos with sensitivity low ##############
######################################################

if (sensitivity == "low") & (intensity == "low") & (duration == "short"):
    output.risk[output['level'] <0.06] = 'low'
    output.risk[(output['level'] >= 0.06) & (output['level'] < 0.075)] = 'medium'
    output.risk[(output['level'] >= 0.075) & (output['level'] < 0.09)] = 'high'
    output.risk[(output['level'] >= 0.09)] = 'very high'
    
if (sensitivity == "low") & (intensity == "low") & (duration == "medium"):
    output.risk[output['level'] <0.055] = 'low'
    output.risk[(output['level'] >= 0.055) & (output['level'] < 0.07)] = 'medium'
    output.risk[(output['level'] >= 0.07) & (output['level'] < 0.085)] = 'high'
    output.risk[(output['level'] >= 0.085)] = 'very high'
    
if (sensitivity == "low") & (intensity == "low") & (duration == "long"):
    output.risk[output['level'] <0.05] = 'low'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.065)] = 'medium'
    output.risk[(output['level'] >= 0.065) & (output['level'] < 0.08)] = 'high'
    output.risk[(output['level'] >= 0.08)] = 'very high'
    
if (sensitivity == "low") & (intensity == "mod") & (duration == "short"):
    output.risk[output['level'] <0.055] = 'low'
    output.risk[(output['level'] >= 0.055) & (output['level'] < 0.07)] = 'medium'
    output.risk[(output['level'] >= 0.07) & (output['level'] < 0.085)] = 'high'
    output.risk[(output['level'] >= 0.085)] = 'very high'
    
if (sensitivity == "low") & (intensity == "mod") & (duration == "medium"):
    output.risk[output['level'] <0.05] = 'low'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.065)] = 'medium'
    output.risk[(output['level'] >= 0.065) & (output['level'] < 0.08)] = 'high'
    output.risk[(output['level'] >= 0.08)] = 'very high'
    
if (sensitivity == "low") & (intensity == "mod") & (duration == "long"):
    output.risk[output['level'] <0.045] = 'low'
    output.risk[(output['level'] >= 0.045) & (output['level'] < 0.06)] = 'medium'
    output.risk[(output['level'] >= 0.06) & (output['level'] < 0.075)] = 'high'
    output.risk[(output['level'] >= 0.075)] = 'very high'    
    
if (sensitivity == "low") & (intensity == "high") & (duration == "short"):
    output.risk[output['level'] <0.050] = 'low'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.065)] = 'medium'
    output.risk[(output['level'] >= 0.065) & (output['level'] < 0.080)] = 'high'
    output.risk[(output['level'] >= 0.080)] = 'very high'
    
if (sensitivity == "low") & (intensity == "high") & (duration == "medium"):
    output.risk[output['level'] <0.05] = 'low'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.060)] = 'medium'
    output.risk[(output['level'] >= 0.060) & (output['level'] < 0.08)] = 'high'
    output.risk[(output['level'] >= 0.08)] = 'very high'
    
if (sensitivity == "low") & (intensity == "high") & (duration == "long"):
    output.risk[output['level'] <0.040] = 'low'
    output.risk[(output['level'] >= 0.040) & (output['level'] < 0.06)] = 'medium'
    output.risk[(output['level'] >= 0.06) & (output['level'] < 0.075)] = 'high'
    output.risk[(output['level'] >= 0.075)] = 'very high' 

    
    
######################################################
######  All combos with sensitivity high ##############
######################################################

if (sensitivity == "high") & (intensity == "low") & (duration == "short"):
    output.risk[output['level'] <0.055] = 'low'
    output.risk[(output['level'] >= 0.055) & (output['level'] < 0.065)] = 'medium'
    output.risk[(output['level'] >= 0.065) & (output['level'] < 0.75)] = 'high'
    output.risk[(output['level'] >= 0.075)] = 'very high'
    
if (sensitivity == "high") & (intensity == "low") & (duration == "medium"):
    output.risk[output['level'] <0.05] = 'low'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.065)] = 'medium'
    output.risk[(output['level'] >= 0.065) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.070)] = 'very high'
    
if (sensitivity == "high") & (intensity == "low") & (duration == "long"):
    output.risk[output['level'] <0.045] = 'low'
    output.risk[(output['level'] >= 0.045) & (output['level'] < 0.06)] = 'medium'
    output.risk[(output['level'] >= 0.06) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.07)] = 'very high'
    
if (sensitivity == "high") & (intensity == "mod") & (duration == "short"):
    output.risk[output['level'] <0.05] = 'low'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.065)] = 'medium'
    output.risk[(output['level'] >= 0.065) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.070)] = 'very high'
    
if (sensitivity == "high") & (intensity == "mod") & (duration == "medium"):
    output.risk[output['level'] <0.045] = 'low'
    output.risk[(output['level'] >= 0.045) & (output['level'] < 0.06)] = 'medium'
    output.risk[(output['level'] >= 0.06) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.07)] = 'very high'
    
if (sensitivity == "high") & (intensity == "mod") & (duration == "long"):
    output.risk[output['level'] <0.040] = 'low'
    output.risk[(output['level'] >= 0.04) & (output['level'] < 0.055)] = 'medium'
    output.risk[(output['level'] >= 0.055) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.070)] = 'very high'    
    
if (sensitivity == "high") & (intensity == "high") & (duration == "short"):
    output.risk[output['level'] <0.045] = 'low'
    output.risk[(output['level'] >= 0.045) & (output['level'] < 0.06)] = 'medium'
    output.risk[(output['level'] >= 0.06) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.07)] = 'very high'
    
if (sensitivity == "high") & (intensity == "high") & (duration == "medium"):
    output.risk[output['level'] <0.040] = 'low'
    output.risk[(output['level'] >= 0.04) & (output['level'] < 0.055)] = 'medium'
    output.risk[(output['level'] >= 0.055) & (output['level'] < 0.07)] = 'high'
    output.risk[(output['level'] >= 0.070)] = 'very high'      
    
if (sensitivity == "high") & (intensity == "high") & (duration == "long"):
    output.risk[output['level'] <0.035] = 'low'
    output.risk[(output['level'] >= 0.035) & (output['level'] < 0.05)] = 'medium'
    output.risk[(output['level'] >= 0.05) & (output['level'] < 0.065)] = 'high'
    output.risk[(output['level'] >= 0.065)] = 'very high' 

output    

//anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launche

,time,level,risk
0,+8 hours,0.036,medium
1,+16 hours,0.017,low
2,+24 hours,0.056,high
3,+32 hours,0.033,low
4,+40 hours,0.014,low
5,+48 hours,0.059,high


In [235]:
#in the app, there needs to be one list of dictionaries that is used to fill in the table output
#get this formatting perfect so it works in the app
ozone = []
for i in range(len(output)):
    my_dict = {output.columns[0]:output.iloc[i][0], output.columns[1]:output.iloc[i][1], output.columns[2]:output.iloc[i][2]}
    ozone.append(my_dict)
ozone

[{'time': '+8 hours', 'level': 0.03562997137830273, 'risk': 'medium'},
 {'time': '+16 hours', 'level': 0.01690378803817274, 'risk': 'low'},
 {'time': '+24 hours', 'level': 0.05642207484170712, 'risk': 'high'},
 {'time': '+32 hours', 'level': 0.03312663780266793, 'risk': 'medium'},
 {'time': '+40 hours', 'level': 0.013920087103398902, 'risk': 'low'},
 {'time': '+48 hours', 'level': 0.058528990993461524, 'risk': 'high'}]